In [ ]:
import os

import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import random

import logging
import time
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import load_model

import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#file_path = '/content/drive/MyDrive/boilers_drive/normalized_df.csv'
train_csv_path = '/content/drive/MyDrive/boilers_drive/train_df.csv'
val_csv_path = '/content/drive/MyDrive/boilers_drive/val_df.csv'
test_csv_path = '/content/drive/MyDrive/boilers_drive/test_df.csv'

Mounted at /content/drive


In [ ]:
# Read the CSV file
#train_data = pd.read_csv(train_csv_path)
#val_data = pd.read_csv(val_csv_path)
test_data = pd.read_csv(test_csv_path)

# Here we only need test data

In [ ]:
#print(train_data)
#print(val_data)
print(test_data)

                        time house_id  normalized_blr_mod_lvl  \
0        2023-03-23 00:00:00  home101            2.975967e-09   
1        2023-03-23 00:01:00  home101            1.983978e-09   
2        2023-03-23 00:02:00  home101            1.322652e-09   
3        2023-03-23 00:03:00  home101            8.817681e-10   
4        2023-03-23 00:04:00  home101            5.878454e-10   
...                      ...      ...                     ...   
1542235  2023-04-30 23:55:00   home95            3.051793e-19   
1542236  2023-04-30 23:56:00   home95            2.034529e-19   
1542237  2023-04-30 23:57:00   home95            1.356353e-19   
1542238  2023-04-30 23:58:00   home95            9.042351e-20   
1542239  2023-04-30 23:59:00   home95            6.028234e-20   

         normalized_absorption  normalized_insulation  normalized_t_r_set  \
0                     0.278222               0.658129            0.711111   
1                     0.278081               0.659089            

In [ ]:
# load pre-prepared random order of houses
random_order_houses = pd.read_csv('/content/drive/MyDrive/boilers_drive/random_order_houses.csv')

In [ ]:
# Different houses in data
houses = random_order_houses['house_id'].unique().tolist()
print("Different houses in data:")
print(houses)
print("Number of different houses:")
print(len(houses))

Different houses in data:
['home9', 'home114', 'home5', 'home89', 'home17', 'home63', 'home2', 'home101', 'home14', 'home95', 'home111', 'home67', 'home77', 'home43', 'home86', 'home90', 'home47', 'home110', 'home93', 'home53', 'home34', 'home51', 'home106', 'home46', 'home79', 'home55', 'home65', 'home13']
Number of different houses:
28


**Defining the transformer model for proper loading**

In [ ]:
# !! Defining the transformer !!

# positional encoding layer
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)


# positional embedding layer
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Dense(d_model)  # Project input to d_model dimension
    #self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

# Attention layers
# These are all identical except for how the attention is configured

# base attention layer
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()


# cross attention layer
# (at the center of the Transformer is the cross-attention layer, it connects the encoder and decoder)
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(query=x, key=context, value=context, return_attention_scores=True)
    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x


# global self attention layer
# (it is responsible for processing the context sequence, and propagating information along its length)
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(query=x, value=x, key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x


# causal self attention layer
# (it does a similar job as the global self attention layer, for the output sequence)
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(query=x, value=x, key=x, use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x


# feed forward network
# (the transformer also includes this point-wise feed-forward network in both the encoder and decoder)
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x


# encoder layer
# (the encoder contains a stack of N encoder layers. Where each EncoderLayer contains
#   a GlobalSelfAttention and FeedForward layer)
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x


# The Encoder
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model, num_heads=num_heads, dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)


  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.
    # Add dropout.
    x = self.dropout(x)
    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.


# decoder layer
# (the decoder's stack is slightly more complex, with each DecoderLayer containing
#   a CausalSelfAttention, a CrossAttention, and a FeedForward layer)
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, *, d_model, num_heads, dff, dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)
    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores
    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x


# The Decoder
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size, d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads, dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None


  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)
    x = self.dropout(x)
    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores
    # The shape of x is (batch_size, target_seq_len, d_model).
    return x


# The Transformer
@tf.keras.utils.register_keras_serializable()
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()

    self.input_proj = tf.keras.layers.Dense(d_model)  # Project input to the model dimension

    self.encoder = Encoder(num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size, dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size, dropout_rate=dropout_rate)

    #self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    self.final_layer = tf.keras.layers.Dense(1)


  def call(self, inputs):  # Expecting only one input
    x = inputs
    #x = self.input_proj(x) # because we get n values for every 1 output value
    context = self.encoder(x)
    x = self.decoder(x, context)
    logits = self.final_layer(x)
    try:
      del logits._keras_mask
    except AttributeError:
      pass
    return logits


**Preparing the data**

In [ ]:
# the categories for prediction
final_category = 'blr_mod_lvl'
prediction_categories = ['blr_mod_lvl', 'absorption', 'insulation', 't_r_set', 't_out']
normalized_categories = ['normalized_blr_mod_lvl', 'normalized_absorption', 'normalized_insulation', 'normalized_t_r_set', 'normalized_t_out']

In [ ]:
# Getting the trained model from saved file

def load_transformer_model(input_categories, num_of_houses):
    categories_used = '_'.join(input_categories)
    houses_used = str(num_of_houses)

    path_to_file = '/content/drive/Othercomputers/My_Laptop/code/models/'
    model_name = 'transfomrer_'+categories_used+"_houses_"+houses_used
    # !!! due to typing error the names have 'transfomrer' instead of 'transformer' !!
    path_to_saved_model = path_to_file + model_name + '.keras'
    path_to_history = path_to_file + model_name + '_history.pkl'

    # Loading from store
    model = tf.keras.models.load_model(path_to_saved_model, custom_objects={'Transformer': Transformer})
    with open(path_to_history, 'rb') as f:
        history = pickle.load(f)

    return model, history


def load_lstm_model(input_categories, num_of_houses):
    categories_used = '_'.join(input_categories)
    houses_used = str(num_of_houses)

    path_to_file = '/content/drive/Othercomputers/My_Laptop/code/models/'
    model_name = 'lstm_'+categories_used+"_houses_"+houses_used

    path_to_saved_model = path_to_file + model_name + '.keras'
    path_to_history = path_to_file + model_name + '_history.pkl'

    # Loading from store
    model = tf.keras.models.load_model(path_to_saved_model)
    with open(path_to_history, 'rb') as f:
        history = pickle.load(f)

    return model, history


model, history = load_transformer_model(['blr_mod_lvl'], 1)

print(model.summary())
print(history)

TypeError: <class '__main__.Transformer'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': None, 'class_name': 'Transformer', 'config': {'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'Transformer', 'build_config': {'input_shape': [None, 1440, 1]}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 0.0010000000474974513, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name

In [ ]:
model, history = load_transformer_model(['blr_mod_lvl'], 1)

print(model.summary())
print(history)

TypeError: <class '__main__.Transformer'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': None, 'class_name': 'Transformer', 'config': {'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}, 'registered_name': 'Transformer', 'build_config': {'input_shape': [None, 1440, 1]}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 0.0010000000474974513, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name

In [ ]:
# Plotting the training and validation loss and MAE
def training_plots(history):
    # Plotting the training and validation loss and MAE
    plt.figure(figsize=(14, 5))

    # Plot training & validation loss values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend(loc='upper right')

    # Plot training & validation MAE values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mean_absolute_error'], label='Training MAE')
    plt.plot(history.history['val_mean_absolute_error'], label='Validation MAE')
    plt.title('Model Mean Absolute Error')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

In [ ]:
training_plots(history)

In [ ]:
# The necessary functions to get the correct test dataset for predictions !!

# creating sub-lists, each with the data of one day
# the function that gets category data of a house (ordered by date) and separates by day
def separate_into_days(data_list, minutes_per_day=1440):
    # number of days
    num_days = len(data_list) // minutes_per_day
    # the data into a list of sub-lists, each containing one day's data
    separated_data = [
        data_list[i * minutes_per_day:(i + 1) * minutes_per_day]
        for i in range(num_days)
    ]
    return separated_data


# function to "combine" values of categories and separate into sub-lists based on days
def combine_categories(dataset, categories_list):
    # Combine specified categories into lists
    combined_elements = dataset[categories_list].apply(lambda row: row.tolist(), axis=1)
    return combined_elements.tolist()


def prepare_data_2(house_data, input_categories, output_category, minutes_per_day=1440):
    combined_input_data = combine_categories(house_data, input_categories)
    separated_input_data = separate_into_days(combined_input_data, minutes_per_day)
    output = house_data[output_category].values
    separated_output = separate_into_days(output, minutes_per_day)
    # [:, :-1] and [:, 1:] for 2-d arrays
    # [:-1],  and [1:] for 1-d arrays (or lists)
    # all except last day are inputs (for prediction)
    input_data = separated_input_data[:-1]
    # all except first day are the corresponding outputs (from prediction)
    output_data = separated_output[1:]
    return input_data, output_data


# function for getting input and target data
def input_target_split (data, input_categories, num_of_houses):
    filtered_data = data[['house_id', 'time', 'normalized_blr_mod_lvl', 'normalized_absorption', 'normalized_insulation', 'normalized_t_r_set', 'normalized_t_out']].copy()
    input_chosen_categories = []
    for cat in input_categories:
        input_chosen_categories.append('normalized_'+cat)

    # We have 1440 minutes per day
    minutes_per_day = 1440

    # Initialize lists to store input and target pairs
    input_data_list, target_data_list = [], []

    for house_id in houses[:num_of_houses]:
        house_data = filtered_data[filtered_data['house_id'] == house_id]
        house_data = house_data.sort_values(by='time')
        input_data, target_data = prepare_data_2(house_data, input_chosen_categories, 'normalized_blr_mod_lvl')
        input_data_list.append(input_data)
        target_data_list.append(target_data)

    # Combine all houses' data
    input_data = np.concatenate(input_data_list, axis=0)
    target_data = np.concatenate(target_data_list, axis=0)

    return input_data, target_data


def get_split_test_data(test_data, input_categories, num_of_houses):
    input_test, target_test = input_target_split(test_data, input_categories, num_of_houses)
    return input_test, target_test


def get_test_datasets(input_test, target_test):
    batch_size = 8 # Reduce the batch size, in case it helps !

    test_dataset = tf.data.Dataset.from_tensor_slices((input_test, target_test))
    test_dataset = test_dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # The validation_split argument is designed to work with NumPy arrays or TensorFlow tensors,
    # where it can easily split the data based on a fraction.
    # It doesn't directly work with TensorFlow datasets because they handle data differently.
    return test_dataset


def print_test_shapes(input_test, target_test):
    print(input_test.shape)
    print(target_test.shape)
    num_of_categories = input_test.shape[-1]
    print("Number of categories for prediction: "+str(num_of_categories))
    return num_of_categories

In [ ]:
# Get the correct test dataset
def get_test_dataset(test_data, input_categories, num_of_houses):
    print("splitting data")
    input_test, target_test = get_split_test_data(test_data, input_categories, num_of_houses)
    print("getting datasets")
    test_dataset = get_test_datasets(input_test, target_test)
    print("printing shapes and getting num_of_categories")
    num_of_categories = print_test_shapes(input_test, target_test)
    return test_dataset

In [ ]:
# Evaluate the model on the test dataset
test_results = model.evaluate(test_dataset, return_dict=True)
#print(f"Test Loss (MSE): {test_results['loss']}, Test MAE: {test_results['mean_absolute_error']}")

# model.predict() !!! (not model.evaluate)

148/148 [==============================] - 33s 222ms/step - loss: 0.0068 - mean_absolute_error: 0.0314


In [ ]:
test_predictions = model.predict(test_dataset)

148/148 [==============================] - 34s 219ms/step


In [ ]:
#print(test_predictions.shape)
#flattened_predictions = test_predictions.reshape(-1)
#print(flattened_predictions.shape)
#print(flattened_predictions)
#de_scaled_predictions = de_scale(flattened_predictions.reshape(-1, 1), 'blr_mod_lvl')
#print(de_scaled_predictions)

# we get the predictions that correspond to the target_test

#print(target_test.shape)
#flattened_target = target_test.reshape(-1)
#print(flattened_target)
#de_scaled_target = de_scale(flattened_target.reshape(-1, 1), 'blr_mod_lvl')
#print(de_scaled_target)

In [ ]:
def get_test_values(test_values):
  flattened_test_values = test_values.reshape(-1)
  de_scaled_test_values = de_scale(flattened_test_values.reshape(-1, 1), 'blr_mod_lvl')
  final_test_values = de_scaled_test_values.reshape(-1)
  return final_test_values

In [ ]:
# function which randomly selects one day from the original prediction results
def get_random_day(predictions, target):
  random_day_index = random.randint(0, predictions.shape[0] - 1)
  selected_day_prediction = predictions[random_day_index]
  selected_day_target = target[random_day_index]
  selected_day_prediction_final = get_test_values(selected_day_prediction)
  selected_day_target_final = get_test_values(selected_day_target)
  return selected_day_prediction_final, selected_day_target_final

In [ ]:
random_day_prediction, random_day_target = get_random_day(test_predictions, target_test)
print("Random Day Prediction:", random_day_prediction)
print("Random Day Target:", random_day_target)

Random Day Prediction: [ 5.0322704  4.8768334  4.7818475 ... -1.0015002 -1.141188  -1.2381945]
Random Day Target: [19.89303692 18.42637647 14.67591764 ...  1.84059946  1.2270663
  0.8180442 ]


In [ ]:
# function that gives us the error metrics for given pair target, prediction
def get_error_metrics(target, prediction):
  # error values
  error = [t - p for t, p in zip(target, prediction)]
  AE = [abs(e) for e in error] # Absolute Error
  SE = [e ** 2 for e in error] # Squared Error
  APE = [abs((t - p) / t) for t, p in zip(target, prediction)] # Absolute Percentage Error

  # error metrics
  # Mean Absolute Error, gives magnitude of errors without caring for direction
  MAE = np.mean(AE)
  # Mean Squared Error, gives higher weight for larger errors
  MSE = np.mean(SE)
  # Root Mean Squared Error,  it is in the same units as the target variable
  RMSE = np.sqrt(MSE)
  # Mean Absolute Percentage Error, provides a perspective on the size of the error relative to the target values (given as percentage %)
  MAPE = np.mean(APE) * 100
  # R-Squared, statistical measure that represents the proportion of the variance for the target variable that's explained by the model
  # provides an indication of the goodness of fit
  mean_target = np.mean(target)
  diff = [t - mean_target for t in target]
  denominator = [d ** 2 for d in diff]
  R2 = 1 - (sum(SE) / sum(denominator))

  # minimum and maximum errors and values
  minimum_error = np.min(error)
  maximum_error = np.max(error)
  min_pred = np.min(prediction)
  max_pred = np.max(prediction)
  min_target = np.min(target)
  max_target = np.max(target)

  errors = pd.DataFrame({
      'blr_mod_lvl': target,
      'prediction': prediction,
      'error': error,
      'AE': AE,
      'SE': SE,
      'APE': APE
  })

  errors['MAE'] = MAE
  errors['MSE'] = MSE
  errors['RMSE'] = RMSE
  errors['MAPE'] = MAPE
  errors['R2'] = R2
  errors['minimum_error'] = minimum_error
  errors['maximum_error'] = maximum_error
  errors['minimum_prediction'] = min_pred
  errors['maximum_prediction'] = max_pred
  errors['minimum_target'] = min_target
  errors['maximum_target'] = max_target

  return errors


In [ ]:
target = get_test_values(target_test)
prediction = get_test_values(test_predictions)

errors = get_error_metrics(target, prediction)

print("\n Table with errors")
print(errors)

<ipython-input-38-470568afdeb7>:7: RuntimeWarning: divide by zero encountered in scalar divide
  APE = [abs((t - p) / t) for t, p in zip(target, prediction)] # Absolute Percentage Error
<ipython-input-38-470568afdeb7>:7: RuntimeWarning: overflow encountered in scalar divide
  APE = [abs((t - p) / t) for t, p in zip(target, prediction)] # Absolute Percentage Error
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)



 Table with errors
          blr_mod_lvl  prediction     error        AE        SE           APE  \
0        2.048427e-08    0.171338 -0.171338  0.171338  0.029357  8.364362e+06   
1        1.365618e-08    0.158856 -0.158856  0.158856  0.025235  1.163255e+07   
2        9.104121e-09    0.166520 -0.166520  0.166520  0.027729  1.829066e+07   
3        6.069414e-09    0.190216 -0.190216  0.190216  0.036182  3.134002e+07   
4        4.046276e-09    0.202335 -0.202335  0.202335  0.040939  5.000519e+07   
...               ...         ...       ...       ...       ...           ...   
1702075  6.794076e-19    0.520067 -0.520067  0.520067  0.270470  7.654714e+17   
1702076  4.529384e-19    0.500196 -0.500196  0.500196  0.250196  1.104336e+18   
1702077  3.019589e-19    0.486260 -0.486260  0.486260  0.236449  1.610352e+18   
1702078  2.013060e-19    0.461625 -0.461625  0.461625  0.213097  2.293150e+18   
1702079  1.342040e-19    0.394146 -0.394146  0.394146  0.155351  2.936917e+18   

       

In [ ]:
# Now the errors, error metrics for the random day
errors_day = get_error_metrics(random_day_target, random_day_prediction)

print("\n Table with errors for random day")
print(errors_day)


 Table with errors for random day
      blr_mod_lvl  prediction      error         AE          SE       APE  \
0       19.893037    5.032270  14.860766  14.860766  220.842381  0.747034   
1       18.426376    4.876833  13.549543  13.549543  183.590116  0.735334   
2       14.675918    4.781847   9.894070   9.894070   97.892624  0.674170   
3       10.114501    4.724797   5.389704   5.389704   29.048908  0.532869   
4        6.743000    4.799429   1.943571   1.943571    3.777470  0.288235   
...           ...         ...        ...        ...         ...       ...   
1435     4.141349   -0.665097   4.806446   4.806446   23.101924  1.160599   
1436     2.760899   -0.907946   3.668845   3.668845   13.460422  1.328859   
1437     1.840599   -1.001500   2.842100   2.842100    8.077531  1.544116   
1438     1.227066   -1.141188   2.368254   2.368254    5.608629  1.930013   
1439     0.818044   -1.238194   2.056239   2.056239    4.228117  2.513603   

           MAE       MSE       RMSE  MAP

<ipython-input-38-470568afdeb7>:7: RuntimeWarning: divide by zero encountered in scalar divide
  APE = [abs((t - p) / t) for t, p in zip(target, prediction)] # Absolute Percentage Error


In [ ]:
# get plots with the error metrics
def plot_error_metrics(errors):
    plt.figure(figsize=(14, 10))
    # we will make n plots
    ploting_items = errors
    plot_list = [['blr_mod_lvl', 'prediction', 'minimum_prediction', 'maximum_prediction', 'minimum_target', 'maximum_target'],
                  ['error', 'minimum_error', 'maximum_error'], ['AE', 'MAE', 'RMSE'], ['SE', 'MSE']]
    n = len(plot_list)
    for i in range(n):
        to_plot = plot_list[i]
        plt.subplot(1, n+1, i)
        for j in range(1, len(to_plot)+1):
            to_plot_name = to_plot[j-1]
            plt.plot(ploting_items[to_plot_name], label=to_plot_name)
        #plt.title()
        plt.xlabel('Time')
        #plt.ylabel()
        plt.legend(loc='upper right')